# Kusto Analysis

The aim of this notebook is to provide an example of analysing security data from a custom
[Kusto aka Azure Data Explorer (ADE) cluster](https://docs.microsoft.com/en-us/azure/data-explorer/data-explorer-overview).

Kusto/ADE is a fast and highly scalable data exploration service for log and telemetry data, hosted in Azure - and is used across Microsoft
for analysing huge datasets of this sort.

### Example Data

We use the [Open Threat Research Forge Mordor Security Datasets](https://github.com/OTRF/Security-Datasets/), and assume that these have been
loaded already into a Kusto/ADE cluster that you control.

___See: [./Kusto-Ingest.ipynb](./Kusto-Ingest.ipynb) for details on data retrieval, prep and loading___.

In [1]:
from datetime import timedelta
import os

from msticpy.common.pkg_config import refresh_config
from msticpy.data.data_providers import QueryProvider

## Configuration

In [2]:
# Replace these with your own.

KUSTO_CLUSTER = 'msticpykustodemo.ukwest'
KUSTO_DATABASE = 'MsticPyKustoDemo'

# Write and load a temporary MSTICPY Config.
# Edit here, or alternatively, put the necessary config into your primary msticpyconfig.yaml

MSTICPY_KUSTO_CONFIG = f"""
Azure:
  auth_methods:
  - cli
  cloud: global
DataProviders:
  Kusto-MSTICPY:
    Args:
      Cluster: "https://{KUSTO_CLUSTER}.kusto.windows.net"
      IntegratedAuth: true
"""

kusto_config_path = os.path.abspath(os.path.join('.', 'msticpyconfig-kusto-analysis-temp.yaml'))

try:
    os.environ['MSTICPYCONFIG'] = kusto_config_path
    with open(kusto_config_path, 'w') as f:
        f.write(MSTICPY_KUSTO_CONFIG)
    refresh_config()
finally:
    os.unlink(kusto_config_path)

## Basic Usage

In [3]:
# Load the Kusto Query Provider and connect.
# The specified cluster must exist within your config.

kusto_prov = QueryProvider('Kusto')
kusto_prov.connect(cluster=KUSTO_CLUSTER, database=KUSTO_DATABASE)

Please wait. Loading Kqlmagic extension...done
Connecting... 

connected


In [4]:
# If everything is working correctly, you should have some data available to query!

event1_count = kusto_prov.exec_query('Event1 | count').iloc[0].Count
print('[+] Event1 #Records:', event1_count)

[+] Event1 #Records: 1240


## Mimikatz Process Herpaderping Example

See https://github.com/jxy-s/herpaderping for details, but briefly, the technique aims to bypass anti-virus detections by
creating a suspended process from a legitimate/signed binary - then replacing this image with an illegitimate one prior to
continuing execution.

We'll look at an example contained within the Mordor data.

In [5]:
# Search for processes created with the usefully named "ProcessHerpaderping.exe" Image:

herpaderping_processes = kusto_prov.exec_query("""
    Event1
    | where Image has "ProcessHerpaderping.exe"
    | project Timestamp, Hostname, ProcessId, Image, CommandLine
""")

herpaderping_processes.head()

,Timestamp,Hostname,ProcessId,Image,CommandLine
0,2020-10-27 08:28:57.062000+00:00,WORKSTATION5,10164,C:\Users\wardog\Desktop\ProcessHerpaderping.exe,ProcessHerpaderping.exe mimikatz.exe wardog.e...


In [6]:
# Take the first such process, and retrieve the child processes created by it within a 30s time window:

target_timestamp, target_hostname, target_processid = (
    herpaderping_processes.iloc[0].Timestamp,
    herpaderping_processes.iloc[0].Hostname,
    herpaderping_processes.iloc[0].ProcessId
)

target_timestamp_range = (target_timestamp.isoformat(), (target_timestamp + timedelta(seconds=30)).isoformat())

child_processes = kusto_prov.exec_query(f"""
    Event1
    | where Timestamp >= datetime({target_timestamp_range[0]}) and Timestamp < datetime({target_timestamp_range[1]})
    | where Hostname == "{target_hostname}"
    | where ParentProcessId == "{target_processid}"
    | order by Timestamp asc
""")

child_processes.head()

,ProcessId,ProcessGuid,Task,Version,Domain,Keywords,AccountName,SourceName,UserID,Hostname,...,LogonId,Timestamp,Port,Tags,Host,ProcessID,ERROR_EVT_UNRESOLVED,Type,TimeCreated,Level
0,8924,{39e4a257-1289-5f98-482d-000000000700},1,NaN,,0x8000000000000000,,Microsoft-Windows-Sysmon,,WORKSTATION5,...,0xc61d9,2020-10-27 08:28:57.420000+00:00,NaN,,,NaN,NaN,,2020-10-27 08:28:57.420000+00:00,4.0


In [7]:
# Now, for the first child process, retrieve the loaded images together with a few of their properties.

child_process_id = child_processes.iloc[0].ProcessId

child_process_image_loads = kusto_prov.exec_query(f"""
    Event7
    | where Timestamp >= datetime({target_timestamp_range[0]}) and Timestamp < datetime({target_timestamp_range[1]})
    | where Hostname == "{target_hostname}"
    | where ProcessId == {child_process_id}
    | order by Timestamp asc
    | project Timestamp, ProcessId, ImageLoaded, Signature, Description
""")

child_process_image_loads.head()

,Timestamp,ProcessId,ImageLoaded,Signature,Description
0,2020-10-27 08:28:57.556000+00:00,8924,C:\Users\wardog\Desktop\wardog.exe,Microsoft Windows,mimikatz for Windows
1,2020-10-27 08:28:57.557000+00:00,8924,C:\Windows\System32\ntdll.dll,Microsoft Windows,NT Layer DLL
2,2020-10-27 08:28:57.557000+00:00,8924,C:\Windows\System32\kernel32.dll,Microsoft Windows,Windows NT BASE API Client DLL
3,2020-10-27 08:28:57.557000+00:00,8924,C:\Windows\System32\KernelBase.dll,Microsoft Windows,Windows NT BASE API Client DLL
4,2020-10-27 08:28:57.585000+00:00,8924,C:\Windows\System32\sechost.dll,Microsoft Windows,Host for SCM/SDDL/LSA Lookup APIs


## PurpleSharp Process Injection Example

Let's explore Sysmon Event 8, which records Remote Thread Creations - whereby one process creates a thread within another.

In [8]:
remote_process_creations = kusto_prov.exec_query(f"""
    Event8
    | summarize Count=count() by SourceImage, TargetImage
    | order by Count desc
""")

remote_process_creations.head()

,SourceImage,TargetImage,Count
0,C:\Windows\System32\csrss.exe,C:\Windows\System32\svchost.exe,197
1,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,C:\Windows\System32\notepad.exe,89
2,C:\Windows\System32\csrss.exe,C:\Windows\System32\wbem\WmiPrvSE.exe,4
3,C:\Windows\System32\csrss.exe,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,3
4,C:\Windows\System32\csrss.exe,C:\Windows\System32\spoolsv.exe,3


In [9]:
# csrss.exe creates a lot of remote threads, so let's filter that:

remote_process_creations = kusto_prov.exec_query(f"""
    Event8
    | where SourceImage !has "csrss.exe"
    | summarize Count=count() by SourceImage, TargetImage
    | order by Count desc
""")

remote_process_creations.head()

,SourceImage,TargetImage,Count
0,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,C:\Windows\System32\notepad.exe,89
1,C:\Windows\System32\dwm.exe,C:\Windows\System32\csrss.exe,3
2,C:\Users\wardog\Desktop\PurpleSharp.exe,C:\Windows\System32\notepad.exe,1
3,C:\Program Files\Internet Explorer\iexplore.exe,&lt;unknown process&gt;,1
4,C:\Windows\System32\wuauclt.exe,&lt;unknown process&gt;,1


In [10]:
# Now, consider "PurpleSharp.exe", which looks interesting:

remote_process_creations = kusto_prov.exec_query(f"""
    Event8
    | where SourceImage has "PurpleSharp.exe"
    | take 5
    | project Timestamp, Hostname, SourceProcessId, SourceImage, TargetProcessId, TargetImage
""")

remote_process_creations.head()

,Timestamp,Hostname,SourceProcessId,SourceImage,TargetProcessId,TargetImage
0,2020-10-23 03:12:04.474000+00:00,WORKSTATION5,8972,C:\Users\wardog\Desktop\PurpleSharp.exe,9908,C:\Windows\System32\notepad.exe


In [11]:
# Now, for the first child process, retrieve any sub-processes that it itself created.

target_timestamp, target_hostname, target_processid = (
    remote_process_creations.iloc[0].Timestamp,
    remote_process_creations.iloc[0].Hostname,
    remote_process_creations.iloc[0].TargetProcessId
)

target_timestamp_range = (target_timestamp.isoformat(), (target_timestamp + timedelta(seconds=60)).isoformat())

child_process_image_loads = kusto_prov.exec_query(f"""
    Event1
    | where Timestamp >= datetime({target_timestamp_range[0]}) and Timestamp < datetime({target_timestamp_range[1]})
    | where Hostname == "{target_hostname}"
    | where ParentProcessId == {target_processid}
    | order by Timestamp asc
    | project Timestamp, Hostname, ParentProcessId, ProcessId, Image, CommandLine
""")

child_process_image_loads.head()

,Timestamp,Hostname,ParentProcessId,ProcessId,Image,CommandLine
0,2020-10-23 03:12:04.930000+00:00,WORKSTATION5,9908.0,5232,C:\Windows\System32\PING.EXE,"""C:\Windows\System32\ping.exe"" 127.0.0.1 -n 10"
